In [2]:
# all imports up here, rerun this block when adding
import pandas as pd
import numpy as np
import cv2 as cv
from sklearn.model_selection import train_test_split

In [14]:
#assumes dataset in folder
wines   = pd.read_csv("XWines_Test_100_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("XWines_Test_1K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

print(ratings)
print(wines)

     RatingID   UserID  WineID Vintage  Rating                 Date
0        3211  1209683  111478    1959     4.5  2016-08-08 00:50:22
1       27878  1209980  111478    1975     4.0  2018-08-12 17:09:39
2       31227  1258705  111478    1975     5.0  2014-11-16 19:52:38
3       41946  1139706  111478    1979     5.0  2014-12-22 02:30:15
4       61700  1240747  111478    1982     4.5  2019-10-21 02:01:10
..        ...      ...     ...     ...     ...                  ...
995  20871491  1006657  102055    N.V.     3.5  2019-12-29 13:38:44
996  20871989  1160496  160271    N.V.     4.0  2015-01-07 12:39:36
997  20875704  1055576  102055    N.V.     3.5  2020-11-03 19:14:49
998  20885463  1004369  102055    N.V.     3.5  2020-08-29 01:26:13
999  20889646  1243678  102055    N.V.     3.5  2019-09-04 18:15:31

[1000 rows x 6 columns]
    WineID                                  WineName          Type  \
0   100062                             Origem Merlot           Red   
1   100191         

In [15]:
def read():
    ##assumes dataset in folder
    wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
    ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
    return wines,ratings

In [16]:

wine_type = input("Enter the type of wine (e.g., red, white, rosé, sparkling): ").lower()
wine_country = input("Enter the country of the wine (e.g., Brazil, New Zealand): ").lower()
wine_body = input("Enter the body of the wine(e.g. ligth-bodied, medium-bodied, full-bodied): ").lower()

filtered_wines = wines.copy()

filtered_wines = wines[(wines['Type'].str.lower() == wine_type) 
                       & (wines['Country'].str.lower() == wine_country)
                       & (wines['Body'].str.lower() == wine_body)]

if filtered_wines.empty:
    filtered_wines = wines[(wines['Type'].str.lower() == wine_type)
                           & (wines['Country'].str.lower() == wine_country)]
    if not filtered_wines.empty:
        print(f"No '{wine_type}' wines in country '{wine_country}' found with body '{wine_body}'")
        print(f"Showing all '{wine_type}' wines from country '{wine_country}' instead.")
    else:
        filtered_wines = wines[wines['Type'].str.lower() == wine_type]
        if not filtered_wines.empty:
            print(f"No '{wine_type}' wines in country '{wine_country}'. Showing all '{wine_type}'instead")
        else:
            print(f"No wines found for the type '{wine_type}'.")
else:
    print(f"Showing '{wine_type}' wines from '{wine_country}'.")

if not filtered_wines.empty:
    print(filtered_wines)
else:
    print("No wines found that match your preferences.")

No 'red' wines in country 'brazil' found with body 'light-bodied'
Showing all 'red' wines from country 'brazil' instead.
   WineID       WineName Type      Elaborate      Grapes  \
0  100062  Origem Merlot  Red  Varietal/100%  ['Merlot']   

                                           Harmonize   ABV         Body  \
0  ['Beef', 'Lamb', 'Veal', 'Grilled', 'Pizza', '...  13.0  Full-bodied   

  Acidity Code Country  RegionID         RegionName  WineryID    WineryName  \
0  Medium   BR  Brazil      1002  Vale dos Vinhedos     10014  Casa Valduga   

                         Website  \
0  http://www.casavalduga.com.br   

                                            Vintages  
0  [2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...  


In [17]:
filtered_reviews = ratings[ratings['WineID'].isin(filtered_wines['WineID'])]

# Merge filtered_wines and filtered_reviews on 'WineID'
merged_df = pd.merge(filtered_wines, filtered_reviews, on='WineID', how='inner')

# Identify columns that belong to the wines and ratings DataFrames
wine_columns = ['WineID', 'Type', 'Country', 'Body']  
rating_columns = ['RatingID', 'WineID', 'UserID', 'Rating']  

# Split the merged DataFrame back into wines and ratings
filtered_wines_df = merged_df[wine_columns].drop_duplicates()  # Drop duplicates to get unique wine data
filtered_reviews_df = merged_df[rating_columns]

print(filtered_wines.head(10))
print(filtered_reviews.head(10))
print(filtered_wines_df.head(10))
print(filtered_reviews_df.head(10))


   WineID       WineName Type      Elaborate      Grapes  \
0  100062  Origem Merlot  Red  Varietal/100%  ['Merlot']   

                                           Harmonize   ABV         Body  \
0  ['Beef', 'Lamb', 'Veal', 'Grilled', 'Pizza', '...  13.0  Full-bodied   

  Acidity Code Country  RegionID         RegionName  WineryID    WineryName  \
0  Medium   BR  Brazil      1002  Vale dos Vinhedos     10014  Casa Valduga   

                         Website  \
0  http://www.casavalduga.com.br   

                                            Vintages  
0  [2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...  
     RatingID   UserID  WineID Vintage  Rating                 Date
199   2644207  2037611  100062    2010     5.0  2018-07-02 02:32:16
554   9655227  1007559  100062    2015     3.5  2017-10-08 01:51:02
673  12205871  1261887  100062    2016     3.0  2017-09-15 14:05:08
726  13944896  1117421  100062    2016     5.0  2018-10-02 20:54:27
728  13980397  1048267  100062    2016     3.5

In [18]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def create_wine_features(wine_data):
    """
    Create and combine wine features (textual, categorical, and numerical) from the wine data.
    
    Args:
    - wine_data: DataFrame containing wine feature data.
    
    Returns:
    - wine_features: Sparse matrix of combined wine features (without weights).
    """
    
    # Make a copy of the wine data to avoid modifying the original DataFrame
    wine_data = wine_data.copy()

    # Preprocessing 'Grapes' and 'Harmonize' columns (convert list to space-separated string)
    wine_data['Grapes'] = wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))
    wine_data['Harmonize'] = wine_data['Harmonize'].apply(lambda x: ' '.join(eval(x)))

    # Function to handle mixed vintages: list or integer
    def convert_to_list(vintages):
        if isinstance(vintages, int):
            return [str(vintages)] 
        elif isinstance(vintages, str):
            try:
                return list(map(str, eval(vintages)))  
            except:
                return [str(vintages)]  
        else:
            return [str(vintages)]  

    #Apply the conversion to the 'Vintages' column and convert to space-separated string
    wine_data['Vintages'] = wine_data['Vintages'].apply(convert_to_list)
    wine_data['Vintages'] = wine_data['Vintages'].apply(lambda x: ' '.join(x))

    #TFIDf vectorization for textual features (Grapes, Harmonize, Vintages)
    tfidf = TfidfVectorizer()
    wine_data_tfidf_grapes = tfidf.fit_transform(wine_data['Grapes'])
    wine_data_tfidf_harmonize = tfidf.fit_transform(wine_data['Harmonize'])
    wine_data_tfidf_vintages = tfidf.fit_transform(wine_data['Vintages'])

    #One-hot encoding for categorical features ('Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName')
    one_hot = OneHotEncoder()
    categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
    wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

    #Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])

    #Combine all features into a single sparse matrix
    wine_features = sp.hstack([
        wine_data_tfidf_grapes, 
        wine_data_tfidf_harmonize, 
        wine_data_tfidf_vintages,
        wine_categorical_onehot, 
        wine_abv_scaled
    ]).tocsr()

    return (wine_features, wine_categorical_onehot, wine_abv_scaled, 
            wine_data_tfidf_grapes, wine_data_tfidf_harmonize, wine_data_tfidf_vintages)

In [19]:
(wine_features, wine_categorical_onehot, wine_abv_scaled, 
        wine_data_tfidf_grapes, wine_data_tfidf_harmonize, wine_data_tfidf_vintages) = create_wine_features(wines)

print(wine_features)

  (0, 46)	1.0
  (0, 99)	0.21955673571950182
  (0, 118)	0.5408368934756286
  (0, 120)	0.2706022462081998
  (0, 126)	0.34079448462595613
  (0, 127)	0.6098072465339026
  (0, 141)	0.3131436732973766
  (0, 197)	0.3284118925850121
  (0, 199)	0.3353503378267026
  (0, 202)	0.2762869497183293
  (0, 203)	0.2875833615002168
  (0, 204)	0.26814666583970825
  (0, 205)	0.2654927698889014
  (0, 206)	0.2628675652670642
  (0, 207)	0.25264167276101906
  (0, 208)	0.25264167276101906
  (0, 209)	0.24524529196446712
  (0, 210)	0.25264167276101906
  (0, 211)	0.2735430893098887
  (0, 212)	0.2875833615002168
  (0, 217)	1.0
  (0, 221)	1.0
  (0, 228)	1.0
  (0, 232)	1.0
  (0, 314)	1.0
  :	:
  (99, 177)	0.4416644271199072
  (99, 181)	0.43290047476286614
  (99, 193)	0.23105892708301426
  (99, 199)	0.2238535793657743
  (99, 200)	0.21033795139328565
  (99, 201)	0.20607208326024323
  (99, 202)	0.1844274946234303
  (99, 203)	0.1919680929951307
  (99, 204)	0.17899367966116686
  (99, 205)	0.17722214690619034
  (99, 206)	0

In [20]:
y_train = filtered_reviews['Rating'].values
x_train = []

for index, row in ratings.iterrows():
    user_ID = row['UserID']
    wine_ID = row['WineID']
    wine_index = wines.index[wines['WineID'] == wine_ID].tolist()[0]
    wine_feature = wine_features[wine_index]
    x_train.append(wine_feature)

x_train = sp.vstack(x_train)


In [28]:
from CB_similarities import CBRecommender
from CB import ContentBasedRecommender

print(CBRecommender)

recommender = ContentBasedRecommender(filtered_reviews, filtered_wines)

x_train_dense = x_train.toarray()
x_train_df = pd.DataFrame(x_train_dense)
print(x_train_df.head(5))

recommender.fit(x_train_df)

# choose a userID from the above filtered_reviews df
recommendations = recommender.recommend(1138214, top_n=5)
print('These recommended wines are similar based on your Type, Country and Body preferences: ')
print(recommendations)


<class 'CB_similarities.CBRecommender'>


ValueError: Found input variables with inconsistent numbers of samples: [1000, 21]

Evaluation

In [1]:
from CB_similarities import CBRecommender
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)
recommender = CBRecommender(train_ratings, wines)
print(train_ratings.shape)
print(wines.shape)
recommender.fit()

user_ids = test_ratings['UserID'].drop_duplicates()

# Store evaluation metrics
precision_list = []
recall_list = []
f1_list = []

# Iterate through each user in the test set
for user_id in user_ids:

    # Your processing code for each user_id goes here
    # Generate recommendations for the user
    recommendations = recommender.recommend(user_id, top_n=5)

    if not recommendations:
        print(f"No recommendations for User {user_id}")
        continue

    # Define relevant wines based on the test ratings for the user
    threshold = 3
    relevant_wines = set(test_ratings[(test_ratings['UserID'] == user_id) & 
                                  (test_ratings['Rating'] >= threshold)]['WineID'])
    # print("Relevant wines: " + str(relevant_wines))
    
    # Extract the recommended wine IDs
    recommended_wine_ids = set(recommendations)
    # print("Recommended Wines: " + str(recommended_wine_ids))
    # Calculate True Positives, False Positives, and False Negatives
    true_positives = len(relevant_wines.intersection(recommended_wine_ids))
    false_positives = len(recommended_wine_ids - relevant_wines)
    false_negatives = len(relevant_wines - recommended_wine_ids)

    # Calculate Precision, Recall, and F1 Score
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0.0  # Avoid division by zero

    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0.0  # Avoid division by zero

    if (precision + recall) > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0  # Avoid division by zero

    # Store the metrics for each user
    # print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    

avg_precision = sum(precision_list) / len(precision_list) if precision_list else 0.0
avg_recall = sum(recall_list) / len(recall_list) if recall_list else 0.0
avg_f1 = sum(f1_list) / len(f1_list) if f1_list else 0.0

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1 Score: {avg_f1}")

NameError: name 'train_test_split' is not defined